# LOGISTIC REGRESSION MODELING

## Modeling Setup

In [1]:
#import libraires and read in data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("/content/merged_health_census_clean (1).csv")

In [2]:
#create obesity categories
df["Obesity_Category"] = pd.qcut(df["Obesity_Pct"], q=3, labels=["Low", "Medium", "High"])
#define features and target
X = df[["Median_Income", "Median_Age", "Poverty_Pct",
        "HS_Grad_Pct", "Bachelors_Pct", "Unemployed_Pct",
        "Pct_White_Alone", "Pct_Black_Alone", "Pct_Asian_Alone",
        "Pct_Hispanic", "Pct_Other"]]
y = df["Obesity_Category"]

In [3]:
#test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80, stratify=y)
#scale features to the same units (z-score)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Building

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


In [16]:
# MULTINOMIAL LOGISTIC REGRESSION MODEL


logit_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logit", LogisticRegression(
        multi_class="multinomial",   # true multinomial softmax
        solver="lbfgs",              # supports penalty='none'
        penalty=None,              # NO regularization
        max_iter=1000,
        n_jobs=-1
    ))
])

In [17]:
# Model fitting
logit_model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('logit',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    n_jobs=-1, penalty=None))])

## Calibration Evaluation Metric

In [18]:
# Predictions for calibration
y_pred_test  = logit_model.predict(X_test)
y_proba_test = logit_model.predict_proba(X_test)

In [19]:
# Function for multiclass calibration
def multiclass_ece(y_true, y_proba, n_bins=10):
    """
    Expected Calibration Error (ECE) for a multiclass classifier
    using the top-predicted probability.
    """
    y_true = np.asarray(y_true)
    y_proba = np.asarray(y_proba)

    confidences = np.max(y_proba, axis=1)
    predictions = np.argmax(y_proba, axis=1)

    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    n = len(y_true)

    for i in range(n_bins):
        bin_lower = bin_edges[i]
        bin_upper = bin_edges[i + 1]

        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        bin_size = np.sum(in_bin)

        if bin_size > 0:
            avg_conf = np.mean(confidences[in_bin])
            avg_acc  = np.mean(predictions[in_bin] == y_true[in_bin])
            ece += (bin_size / n) * np.abs(avg_conf - avg_acc)

    return ece

In [22]:
# ECE Computation
ece_score = multiclass_ece(y_test, y_proba_test, n_bins=20)

print("Expected Calibration Error (ECE):", ece_score)

Expected Calibration Error (ECE): 0.7394992786974887


## Updated Model given ECE (Tunining for best C)

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "logit__C": [0.001,0.01, 0.1, 1, 5, 10]
}

grid = GridSearchCV(
    logit_model,
    param_grid,
    scoring="neg_log_loss",   # proper scoring rule for probabilities
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)
best_model = grid.best_estimator_


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [27]:
# 1. Predict probabilities on the test set
y_proba_best = best_model.predict_proba(X_test)
y_pred_best  = best_model.predict(X_test)

# 2. Compute ECE
ece_best = multiclass_ece(y_test, y_proba_best, n_bins=10)

print("ECE after tuning:", ece_best)


ECE after tuning: 0.7394992786974885


## Model summary

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss

# -----------------------------------
# 1. Get predictions from your model
# -----------------------------------
y_pred = logit_model.predict(X_test)

# -----------------------------------
# 2. Classification report
# -----------------------------------
# If you know the order of your classes:
target_names = ["Low", "Medium", "High"]   # adjust if needed

print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=target_names))

# -----------------------------------
# 3. Accuracy (optional)
# -----------------------------------
acc = accuracy_score(y_test, y_pred)
print(f"\nOverall accuracy: {acc:.3f}")

# -----------------------------------
# 4. Confusion matrix (optional, nice for report)
# -----------------------------------
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm,
    index=[f"True_{c}" for c in target_names],
    columns=[f"Pred_{c}" for c in target_names]
)

print("\nConfusion Matrix:")
print(cm_df)


Classification Report:

              precision    recall  f1-score   support

         Low       0.81      0.81      0.81      1799
      Medium       0.78      0.78      0.78      1828
        High       0.61      0.61      0.61      1796

    accuracy                           0.73      5423
   macro avg       0.73      0.73      0.73      5423
weighted avg       0.73      0.73      0.73      5423


Overall accuracy: 0.733

Confusion Matrix:
             Pred_Low  Pred_Medium  Pred_High
True_Low         1465           11        323
True_Medium        28         1418        382
True_High         321          384       1091
